In [1]:
import placekey as pk
from placekey.api import PlacekeyAPI
import pandas as pd
from ast import literal_eval
import json

In [2]:
placekey_api_key = "Q8J3zrMOuMwyQokDr8x3YX47I7CfXMOQ" # change to env var 
pk_api = PlacekeyAPI(placekey_api_key)

# test placekey api
pk_api.lookup_placekey(latitude=37.7371, longitude=-122.44283)

{'query_id': '0', 'placekey': '@5vg-82n-kzz'}

In [3]:
def get_df_for_api(df, column_map):
  df_for_api = df.rename(columns=column_map)
  cols = list(column_map.values())
  df_for_api = df_for_api[cols]
  df_for_api['iso_country_code'] = 'US'
  df_for_api['query_id'] = df_for_api['query_id'].astype(str)
  return(df_for_api)

In [4]:
# read in data
Blight_Violations_df = pd.read_csv('data/Blight_Violations.csv')
Property_Sales_df = pd.read_csv('data/Property_Sales.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_21308\1233766091.py:2: DtypeWarning: Columns (10,12,17,18,19,39) have mixed types. Specify dtype option on import or set low_memory=False.
  Blight_Violations_df = pd.read_csv('data/Blight_Violations.csv')
C:\Users\cayde\AppData\Local\Temp\ipykernel_21308\1233766091.py:3: DtypeWarning: Columns (8,14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  Property_Sales_df = pd.read_csv('data/Property_Sales.csv')


In [7]:
Blight_Violations_df['zip_code'] = Blight_Violations_df['zip_code'].astype(str)
Blight_Violations_column_map = {"ticket_id": "query_id","mailing_address" : "street_address", "state": "region", "zip_code": "postal_code", "X" : "latitude", "Y" : "longitude", "country" : "iso_country_code", "city" : "city"}
Blight_Violations_for_api = get_df_for_api(Blight_Violations_df, Blight_Violations_column_map)
Blight_Violations_for_api.head()

,query_id,street_address,region,postal_code,latitude,longitude,iso_country_code,city
0,18645,599 King,MI,48202,-83.072568,42.383357,US,Det
1,18646,18604 Appoline,MI,48235,-83.072474,42.383400,US,Det
2,18648,4066 Columbus,MI,48204,-83.115057,42.359928,US,Det
3,18649,3005 Pasada,MI,48238,-83.128037,42.393455,US,Det
4,18650,20211 Westmoreland,MI,48219,-83.134466,42.389668,US,Det


In [8]:
Blight_Violations_data_jsoned = json.loads(Blight_Violations_for_api.to_json(orient="records"))
print("number of records: ", len(Blight_Violations_data_jsoned))
print("example record:")
Blight_Violations_data_jsoned[0]

number of records:  686824
example record:


{'query_id': '18645',
 'street_address': '599 King',
 'region': 'MI',
 'postal_code': '48202',
 'latitude': -83.072568447,
 'longitude': 42.383356721,
 'iso_country_code': 'US',
 'city': 'Det'}

In [52]:
responses_Blight_Violations = pk_api.lookup_placekeys(Blight_Violations_data_jsoned, verbose=True)
placekeys_Blight_Violations = pd.read_json(json.dumps(responses_Blight_Violations), dtype={'query_id':str})
placekeys_Blight_Violations.to_csv('data/placekeys_Blight_Violations.csv', index=False)

2024-03-04 14:13:46,175	INFO	Processed 1000 items
2024-03-04 14:13:46,175	INFO	Processed 1000 items
2024-03-04 14:13:46,176	INFO	Done


In [9]:
Property_Sales_column_map = {"sale_id": "query_id", "address" : "street_address", "X" : "latitude", "Y" : "longitude"}
Property_Sales_for_api = get_df_for_api(Property_Sales_df, Property_Sales_column_map)
Property_Sales_for_api.head()

,query_id,street_address,latitude,longitude,iso_country_code
0,3749837,14006 GLASTONBURY,-83.224446,42.388139,US
1,4210008,19951 ASBURY PARK,-83.210323,42.438587,US
2,4210338,16881 ASBURY PARK,-83.209319,42.415134,US
3,3775822,10322 TIREMAN,-83.161543,42.351916,US
4,3663543,4158 LINCOLN,-83.074014,42.346942,US


In [10]:
Property_Sales_data_jsoned = json.loads(Property_Sales_for_api.to_json(orient="records"))
print("number of records: ", len(Property_Sales_data_jsoned))
print("example record:")
Property_Sales_data_jsoned[0]

number of records:  429730
example record:


{'query_id': '3749837',
 'street_address': '14006 GLASTONBURY',
 'latitude': -83.224445608,
 'longitude': 42.388138734,
 'iso_country_code': 'US'}

In [ ]:
responses_Property_Sales = pk_api.lookup_placekeys(Property_Sales_data_jsoned, verbose=True)
placekeys_Property_Sales = pd.read_json(json.dumps(responses_Property_Sales), dtype={'query_id':str})
placekeys_Property_Sales.to_csv('data/placekeys_Property_Sales.csv', index=False)